## TOPIC - Image classification of Insects

## Importing libraries

In [1]:
import os
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping

%matplotlib inline

2022-12-02 23:40:30.631407: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 23:40:30.770459: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-02 23:40:30.770485: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-02 23:40:30.797316: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-02 23:40:31.572117: W tensorflow/stream_executor/platform/de

## Loading the dataset

In [2]:
my_data_dir = '../img/BeesAndAnts'
test_path = my_data_dir+'/test/'
train_path = my_data_dir+'/train/'

## Rescaling datset

In [3]:
import cv2
import os
path = '../img/BeesAndAnts/train/'
file_path = os.listdir(path)


for i in file_path:
    data = path + i
    img = cv2.imread(data)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(img, (224,224))
    cv2.imwrite(data,image)

In [4]:
path = '../img/BeesAndAnts/test/'
file_path = os.listdir(path)

for i in file_path:
    data = path + i
    img = cv2.imread(data)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(img, (224,224))
    cv2.imwrite(data,image)

In [5]:
import cv2
cv2.imread('../img/BeesAndAnts/train/Image_1.jpg').shape

(224, 224, 3)

## Rearranging Images in respective class folders

In [6]:
train_csv = pd.read_csv("../img/BeesAndAnts/Training_set.csv")
train_csv.label[train_csv["label"]=="bees"] = 0
train_csv.label[train_csv["label"]=="ants"] = 1
train_csv.head()

,filename,label
0,Image_1.jpg,0
1,Image_2.jpg,1
2,Image_3.jpg,1
3,Image_4.jpg,0
4,Image_5.jpg,1


In [7]:
for i in range(len(train_csv.label.unique())):
    try:
        os.mkdir(train_path+str(i))
    except:
        pass
for i,j in zip(train_csv.filename,train_csv.label):
    try:
        os.rename(train_path+i,train_path+str(j)+"/"+i)
    except:
        pass

In [8]:
file_path = os.listdir(test_path)
os.mkdir(test_path+"test")

for i in file_path:
    try:
        os.rename(test_path+i,test_path+"test/"+i)
    except:
        pass

## Now, we are using the image data generator function

In [9]:
image_shape = (224,224,3)
image_gen = ImageDataGenerator(rotation_range=20,
                               width_shift_range=0.10,
                               height_shift_range=0.10,
                               rescale=1/255,
                               shear_range=0.1,
                               zoom_range=0.1,
                               horizontal_flip=True,
                               fill_mode='nearest')

## Building a convolutional neural network model

In [10]:
import tensorflow as tf

In [11]:

batch_size = 64

train_image_gen = image_gen.flow_from_directory(train_path,
                                               target_size= (224,224),
                                                
                                               batch_size=batch_size,
                                               class_mode='categorical')

Found 277 images belonging to 2 classes.


In [12]:
test_image_gen = image_gen.flow_from_directory(test_path,
                                               target_size= (224,224),
                                              
                                               batch_size=batch_size,
                                               class_mode='categorical',shuffle=False)

Found 120 images belonging to 1 classes.


In [13]:
print(train_image_gen.class_indices)
Labels=train_image_gen.class_indices

{'0': 0, '1': 1}


In [14]:
train_image_gen[1][0].shape

(64, 224, 224, 3)

## MobileNetV2

In [15]:

base_model = tf.keras.applications.MobileNetV2(input_shape = (224, 224, 3), include_top = False, weights = "imagenet", classes = 2)

2022-12-02 23:40:34.153453: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/piyush/.local/lib/python3.10/site-packages/cv2/../../lib64:
2022-12-02 23:40:34.153475: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-02 23:40:34.153496: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (piyushLap): /proc/driver/nvidia/version does not exist
2022-12-02 23:40:34.153660: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
base_model.trainable = False

In [17]:
model1 = tf.keras.Sequential([base_model,
                                 tf.keras.layers.GlobalAveragePooling2D(),
                                 tf.keras.layers.Dense(512, activation="relu"),
                                 tf.keras.layers.Dropout(.2),
                                 tf.keras.layers.Dense(256, activation="relu"),
                                 tf.keras.layers.Dropout(.2),
                                 tf.keras.layers.Dense(64, activation="relu"),
                                 tf.keras.layers.Dropout(.2),
                                 tf.keras.layers.Dense(32, activation="relu"),
                                 tf.keras.layers.Dense(2, activation="softmax")                                     
                                ])

In [18]:
base_learning_rate = 0.0001
model1.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model1.fit(train_image_gen, epochs = 50)

Epoch 1/50
5/5 [==============================] - 8s 1s/step - loss: 0.6849 - accuracy: 0.5307
Epoch 2/50
5/5 [==============================] - 6s 1s/step - loss: 0.4551 - accuracy: 0.8159
Epoch 3/50
5/5 [==============================] - 6s 1s/step - loss: 0.2570 - accuracy: 0.9061
Epoch 4/50
5/5 [==============================] - 7s 1s/step - loss: 0.2035 - accuracy: 0.9134
Epoch 5/50
5/5 [==============================] - 6s 1s/step - loss: 0.2474 - accuracy: 0.9061
Epoch 6/50
5/5 [==============================] - 6s 1s/step - loss: 0.1992 - accuracy: 0.9386
Epoch 7/50
5/5 [==============================] - 6s 1s/step - loss: 0.1931 - accuracy: 0.9278
Epoch 8/50
5/5 [==============================] - 6s 1s/step - loss: 0.1785 - accuracy: 0.9386
Epoch 9/50
5/5 [==============================] - 7s 1s/step - loss: 0.1320 - accuracy: 0.9531
Epoch 10/50
5/5 [==============================] - 7s 1s/step - loss: 0.0866 - accuracy: 0.9747
Epoch 11/50
5/5 [==============================] 

In [19]:
predictions = np.argmax(model1.predict(train_image_gen), axis = 1)
predictions

5/5 [==============================] - 6s 990ms/step


array([0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0])

In [20]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(train_image_gen.classes,predictions))

              precision    recall  f1-score   support

           0       0.46      0.46      0.46       142
           1       0.44      0.44      0.44       135

    accuracy                           0.45       277
   macro avg       0.45      0.45      0.45       277
weighted avg       0.45      0.45      0.45       277



## InceptionV3

In [21]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [22]:
base_model = InceptionV3(input_shape = (224, 224, 3), include_top = False)

In [23]:

for layer in base_model.layers:
    layer.trainable = False

In [24]:
from tensorflow.keras.optimizers import RMSprop

x = tf.keras.layers.Flatten()(base_model.output)
x = tf.keras.layers.Dense(2048, activation='relu')(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)

# Add a final sigmoid layer with 1 node for classification output
x = tf.keras.layers.Dense(2, activation='sigmoid')(x)

model2 = tf.keras.models.Model(base_model.input, x)

model2.compile(optimizer = RMSprop(lr=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

In [25]:
inc_history = model2.fit_generator(train_image_gen, epochs = 60)

Epoch 1/60
5/5 [==============================] - 15s 2s/step - loss: 4.3810 - acc: 0.4729
Epoch 2/60
5/5 [==============================] - 12s 2s/step - loss: 1.3040 - acc: 0.7292
Epoch 3/60
5/5 [==============================] - 12s 3s/step - loss: 0.9919 - acc: 0.6895
Epoch 4/60
5/5 [==============================] - 12s 2s/step - loss: 0.5167 - acc: 0.8303
Epoch 5/60
5/5 [==============================] - 12s 2s/step - loss: 0.5913 - acc: 0.7292
Epoch 6/60
5/5 [==============================] - 12s 2s/step - loss: 0.4524 - acc: 0.8159
Epoch 7/60
5/5 [==============================] - 12s 2s/step - loss: 0.3909 - acc: 0.8375
Epoch 8/60
5/5 [==============================] - 12s 2s/step - loss: 0.4157 - acc: 0.8159
Epoch 9/60
5/5 [==============================] - 12s 2s/step - loss: 0.2263 - acc: 0.9242
Epoch 10/60
5/5 [==============================] - 12s 2s/step - loss: 0.2987 - acc: 0.8809
Epoch 11/60
5/5 [==============================] - 12s 2s/step - loss: 0.1374 - acc: 0.94

In [26]:
predictions = np.argmax(model2.predict(train_image_gen), axis = 1)
predictions

5/5 [==============================] - 7s 1s/step


array([0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1])

In [27]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(train_image_gen.classes,predictions))

              precision    recall  f1-score   support

           0       0.52      0.51      0.52       142
           1       0.50      0.50      0.50       135

    accuracy                           0.51       277
   macro avg       0.51      0.51      0.51       277
weighted avg       0.51      0.51      0.51       277



## Test Result CSV

In [28]:
test_csv = pd.read_csv("../img/BeesAndAnts/Testing_set.csv")
test_csv.head()

,filename
0,Image_1.jpg
1,Image_2.jpg
2,Image_3.jpg
3,Image_4.jpg
4,Image_5.jpg


In [29]:
test_csv.label = np.argmax(model1.predict(test_image_gen), axis = 1)

2/2 [==============================] - 3s 1s/step


In [30]:
test_csv.to_csv("test_result_1.csv",index=False)

In [31]:
test_csv.label = np.argmax(model2.predict(test_image_gen), axis = 1)
test_csv.to_csv("test_result_2.csv",index=False)

2/2 [==============================] - 3s 1s/step
